In [1]:
! pip install trulens_eval openai langchain chromadb langchainhub bs4 tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.3/751.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.5/70.5 kB 9.1 

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "Type OPENAI API Key here"

In [2]:
# Imports main tools:
from trulens_eval import TruChain, Tru
tru = Tru()
tru.reset_database()

# Imports from LangChain to build app
import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = CSVLoader(file_path="/content/TCGA_Reports.csv")
docs = loader.load()


In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [5]:
retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [55]:
rag_chain.invoke("What are the histologic type and grade of the lung tumor?")

'The histologic type of the lung tumor is adenocarcinoma, and the grade is poorly differentiated. The tumor size is 5.3 cm.'

In [56]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
import numpy as np

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rag_chain)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance)
    .on_input_output()
)
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.first.steps__.context.first.get_relevant_documents.rets.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In context_relevance_with_cot_reasons, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In context_relevance_with_cot_reasons, input context will be set to __record__.app.first.steps__.context.first.get_relevant_documents.rets .


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [57]:
tru_recorder = TruChain(rag_chain,
    app_id='Kavya Test_Case',
    feedbacks=[f_answer_relevance, f_context_relevance, f_groundedness])

In [104]:
response, tru_record = tru_recorder.with_record(rag_chain.invoke, "Can you provide details about the size and characteristics of the prostate specimen?")

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

In [105]:
json_like = tru_record.layout_calls_as_app()

In [106]:
json_like

Munch({'record_id': 'record_hash_4e387d0f9ab0c776de4b3d3291dc9bdb', 'app_id': 'Kavya Test_Case', 'cost': {'n_requests': 2, 'n_successful_requests': 2, 'n_classes': 0, 'n_tokens': 1193, 'n_stream_chunks': 0, 'n_prompt_tokens': 1132, 'n_completion_tokens': 61, 'cost': 0.001799}, 'perf': {'start_time': '2024-05-02T03:58:50.571394', 'end_time': '2024-05-02T03:58:53.628435'}, 'ts': '2024-05-02T03:58:53.628752', 'tags': '-', 'meta': None, 'main_input': 'Can you provide details about the size and characteristics of the prostate specimen?', 'main_output': 'The prostate specimen received measures 4.8 x 3.5 x 3.5 cm and weighs 34 grams. It is described as tan/brown and firm with an ill-defined discoloration on the right periphery. The specimen also includes attached seminal vesicles and vas deferens.', 'main_error': None, 'calls': [{'stack': [{'path': 'app', 'method': {'obj': {'cls': {'name': 'RunnableSequence', 'module': {'package_name': 'langchain_core.runnables', 'module_name': 'langchain_cor

In [107]:
from ipytree import Tree, Node

def display_call_stack(data):
    tree = Tree()
    tree.add_node(Node('Record ID: {}'.format(data['record_id'])))
    tree.add_node(Node('App ID: {}'.format(data['app_id'])))
    tree.add_node(Node('Cost: {}'.format(data['cost'])))
    tree.add_node(Node('Performance: {}'.format(data['perf'])))
    tree.add_node(Node('Timestamp: {}'.format(data['ts'])))
    tree.add_node(Node('Tags: {}'.format(data['tags'])))
    tree.add_node(Node('Main Input: {}'.format(data['main_input'])))
    tree.add_node(Node('Main Output: {}'.format(data['main_output'])))
    tree.add_node(Node('Main Error: {}'.format(data['main_error'])))

    calls_node = Node('Calls')
    tree.add_node(calls_node)

    for call in data['calls']:
        call_node = Node('Call')
        calls_node.add_node(call_node)

        for step in call['stack']:
            step_node = Node('Step: {}'.format(step['path']))
            call_node.add_node(step_node)
            if 'expanded' in step:
                expanded_node = Node('Expanded')
                step_node.add_node(expanded_node)
                for expanded_step in step['expanded']:
                    expanded_step_node = Node('Step: {}'.format(expanded_step['path']))
                    expanded_node.add_node(expanded_step_node)

    return tree

# Usage
tree = display_call_stack(json_like)
tree

Tree(nodes=(Node(name='Record ID: record_hash_4e387d0f9ab0c776de4b3d3291dc9bdb'), Node(name='App ID: Kavya Tes…

In [108]:
tree

Tree(nodes=(Node(name='Record ID: record_hash_4e387d0f9ab0c776de4b3d3291dc9bdb'), Node(name='App ID: Kavya Tes…

In [109]:
with tru_recorder as recording:
    llm_response = rag_chain.invoke("Can you provide details about the size and characteristics of the prostate specimen?")
display(llm_response)

'The prostate specimen received measures 4.8 x 3.5 x 3.5 cm and weighs 34 grams. It is described as tan/brown and firm with an ill-defined discoloration on the right periphery. The specimen also includes attached seminal vesicles and vas deferens.'

In [110]:
# The record of the app invocation can be retrieved from the `recording`:

rec = recording.get() # use .get if only one record
# recs = recording.records # use .records if multiple

display(rec)

Record(record_id='record_hash_dfa128df0913c615ce8ad8d1d954a3f4', app_id='Kavya Test_Case', cost=Cost(n_requests=2, n_successful_requests=2, n_classes=0, n_tokens=1193, n_stream_chunks=0, n_prompt_tokens=1132, n_completion_tokens=61, cost=0.001799), perf=Perf(start_time=datetime.datetime(2024, 5, 2, 3, 59, 14, 975825), end_time=datetime.datetime(2024, 5, 2, 3, 59, 17, 807196)), ts=datetime.datetime(2024, 5, 2, 3, 59, 17, 807311), tags='-', meta=None, main_input='Can you provide details about the size and characteristics of the prostate specimen?', main_output='The prostate specimen received measures 4.8 x 3.5 x 3.5 cm and weighs 34 grams. It is described as tan/brown and firm with an ill-defined discoloration on the right periphery. The specimen also includes attached seminal vesicles and vas deferens.', main_error=None, calls=[RecordAppCall(stack=[RecordAppCallMethod(path=Lens().app, method=Method(obj=Obj(cls=langchain_core.runnables.base.RunnableSequence, id=139247751783808, init_bind

In [111]:
records, feedback = tru.get_records_and_feedback(app_ids=["Kavya Test_Case"])

records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,relevance,groundedness_measure_with_cot_reasons,context_relevance_with_cot_reasons,relevance_calls,groundedness_measure_with_cot_reasons_calls,context_relevance_with_cot_reasons_calls,latency,total_tokens,total_cost
0,Kavya Test_Case,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_edc07927c4c7b38ac3144e1a30955c57,"""Can you provide a overview of prognostic fact...","""Prognostic factors associated with lung adeno...",-,"{""record_id"": ""record_hash_edc07927c4c7b38ac31...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-02T03:10:25.862230"", ""...",2024-05-02T03:10:29.141587,0.8,0.0,0.8,[{'args': {'prompt': 'Can you provide a overvi...,[{'args': {'source': [[{'page_content': '1. Hi...,[{'args': {'question': 'Can you provide a over...,3,1186,0.001780
1,Kavya Test_Case,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_76f942c1481e34e92289c30512f6d215,"""Can you provide a overview of prognostic fact...","""Prognostic factors associated with lung adeno...",-,"{""record_id"": ""record_hash_76f942c1481e34e9228...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-02T03:10:54.696961"", ""...",2024-05-02T03:10:58.615690,0.8,0.0,0.9,[{'args': {'prompt': 'Can you provide a overvi...,[{'args': {'source': [[{'page_content': '1. Hi...,[{'args': {'question': 'Can you provide a over...,3,1203,0.001814
2,Kavya Test_Case,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_81a28bf51d95950bce3c0ee296faaf7a,"""What are the different specimen collected dur...","""The different specimens collected during the ...",-,"{""record_id"": ""record_hash_81a28bf51d95950bce3...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-02T03:15:26.076201"", ""...",2024-05-02T03:15:30.785749,0.8,0.0,0.8,[{'args': {'prompt': 'What are the different s...,[{'args': {'source': [[{'page_content': 'text:...,[{'args': {'question': 'What are the different...,4,1259,0.001933
3,Kavya Test_Case,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_1180e63c25dbd52c73a041ba30f4993a,"""What are the different specimen collected dur...","""The different specimens collected during the ...",-,"{""record_id"": ""record_hash_1180e63c25dbd52c73a...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-02T03:15:56.927903"", ""...",2024-05-02T03:16:01.429127,0.8,0.0,0.8,[{'args': {'prompt': 'What are the different s...,[{'args': {'source': [[{'page_content': 'text:...,[{'args': {'question': 'What are the different...,4,1280,0.001975
4,Kavya Test_Case,"{""tru_class_info"": {""name"": ""TruChain"", ""modul...",RunnableSequence(langchain_core.runnables.base),record_hash_1fd0d6a28118d94ce5a6f5c35ab5233e,"""Can you outline the targeted therapy options ...","""Targeted therapy options for HER2-positive br...",-,"{""record_id"": ""record_hash_1fd0d6a28118d94ce5a...","{""n_requests"": 2, ""n_successful_requests"": 2, ...","{""start_time"": ""2024-05-02T03:19:05.945223"", ""...",2024-05-02T03:19:09.544501,0.8,0.0,0.8,[{'args': {'prompt': 'Can you outline the targ...,"[{'args': {'source': [[{'page_content': 'HJ, e...",[{'args': {'question': 'Can you outline the ta...,3,1326,0.002016


In [112]:
tru.get_leaderboard(app_ids=["Kavya Test_Case"])

,context_relevance_with_cot_reasons,groundedness_measure_with_cot_reasons,relevance,latency,total_cost
app_id,,,,,
Kavya Test_Case,0.92,0.075,0.91,2.9,0.001726


In [18]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.213s


In [19]:
tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed

Starting dashboard ...
npx: installed 22 in 1.966s

Go to this url and submit the ip given here. your url is: https://new-bottles-wait.loca.lt

  Submit this IP Address: 34.80.199.17



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [22]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()